[View in Colaboratory](https://colab.research.google.com/github/sabumjung/DL-with-TensorFlow/blob/DL-with-TensorFlow/6%EC%9E%A5_2.ipynb)

In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# 학습 parameter 설정
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [0]:
# 신경망 구조 parameter 설정
n_input = 28 
n_steps = 28 
n_hidden = 128 
n_classes = 10 

In [0]:
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

In [0]:
# 가중치와 바이어스 설정
weights = {
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [0]:
# BiRNN 정의
def BiRNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(axis=0, num_or_size_splits=n_steps, value=x)
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [12]:
# 예측값, 손실함수, 최적화방법, 정확도 설정
pred = BiRNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 변수초기화
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
with tf.Session() as sess:
    #변수초기화
    sess.run(init)

    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.733079, Training Accuracy= 0.38281
Iter 2560, Minibatch Loss= 1.555445, Training Accuracy= 0.51562
Iter 3840, Minibatch Loss= 1.079433, Training Accuracy= 0.64062
Iter 5120, Minibatch Loss= 1.080586, Training Accuracy= 0.60938
Iter 6400, Minibatch Loss= 1.092670, Training Accuracy= 0.68750
Iter 7680, Minibatch Loss= 0.843056, Training Accuracy= 0.74219
Iter 8960, Minibatch Loss= 0.781613, Training Accuracy= 0.68750
Iter 10240, Minibatch Loss= 0.541593, Training Accuracy= 0.82812
Iter 11520, Minibatch Loss= 0.499123, Training Accuracy= 0.85938
Iter 12800, Minibatch Loss= 0.540303, Training Accuracy= 0.84375
Iter 14080, Minibatch Loss= 0.508879, Training Accuracy= 0.78125
Iter 15360, Minibatch Loss= 0.575614, Training Accuracy= 0.78906
Iter 16640, Minibatch Loss= 0.303358, Training Accuracy= 0.89062
Iter 17920, Minibatch Loss= 0.222635, Training Accuracy= 0.92969
Iter 19200, Minibatch Loss= 0.392329, Training Accuracy= 0.89844
Iter 20480, Minibatch Loss= 0.30